In [2]:
import llm_core.llm as L
import os

In [3]:
#os.environ['MODEL_PATH'] = "/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf"
LLM_GLOBAL_INSTANCE = L.Llama(
                n_ctx=1000,
                model_path=os.environ['MODEL_PATH'],
                n_gpu_layers=-1, verbose=0,
                # embedding=True
            )

In [4]:
# system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
# user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [5]:
# No Kanji usage, rare katakana usage
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u3040-\u30FF, \u4e00 - \u9faf]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )

{'id': 'chatcmpl-2c2db857-f0fb-4e10-8fb4-17c04df3a11d',
 'object': 'chat.completion',
 'created': 1739819018,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'あがめです。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 5, 'total_tokens': 55}}

In [6]:
#With basic prompt, no character restrictions spamps \u0000 or \uffff
#Good responses with user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
#Talks in any vocab, but keeps sentences in が + です format. :D
# Try telling it to write with spaces between words?
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )

#Really good example sentence: '赤色は炎のように光ってます が 楽しく感じます です。'

{'id': 'chatcmpl-295d7037-805b-4f2e-b5f4-c79980108eea',
 'object': 'chat.completion',
 'created': 1739819019,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '赤色は炎のように光ってます が 楽しく感じます です。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 19, 'total_tokens': 69}}

In [7]:
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [8]:
#only 1 or 2 allowedchar makes for much shorter+concise sentences
#adding more sentences makes them incoherent
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

#grammar works better with Japanese prompts
#English prompts are better without grammar
#Not much variety between responses though
#How does Llama grammar work??!!? Logits?

In [9]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['赤が喜です。',
 '赤が喜です。',
 '赤が火です。',
 '赤が喜です。',
 '赤が力です。',
 '赤が力です。',
 '赤が怒です。',
 '赤が喜です。',
 '赤が怒です。',
 '赤が光です。']

In [10]:
# i and na-adj don't work well
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using adjectives to describe in the [word] が [adjective] form."
user_input = "太陽の景色を描いてください"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle adjective shii end
particle ::= "が"
adjective ::= [\u0000-\uffff]
shii ::= "しい"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [11]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が輝しい。',
 '太があしい。',
 '太が出しい。',
 '太があしい。',
 '太が出しい。',
 '太が出しい。']

In [ ]:
# Needs a good system prompt to use しか at all
# Difficult to get it to use しか normally and not just repeat the user question with it.
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts using the しか～ない Japanese grammar pattern"
user_input = "あんたは普通な人って想像してください。。何をできますか？"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "しか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [26]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ],
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['あなたは普通の人間しか想像できないでしょう。何をできますか？？この質問には、何か特定のスキルや能力があるわけではないので、日常生活の中で必要な基本的な能力が考えられます。例えば、料理を作ったり、掃除を',
 'あなたは普通な人って想像してみてください。何をできるか ila しか想像できませんね。',
 'あんたは普通な人って想像してください。何か特別なことをするしかないのですね。',
 'あんたは普通な人って想像してください。特別なスキルがある場合もあるが、ほとんどは料理するしかありませんな。それも時々しかできませんね。',
 'あなたは普通な人って想像してください。何か特別なことをできるしかありませんな。',
 'あんたは普通な人って想像してください。何か特別なスキルがあるなんて考えられませんか？もしそんなことがないなら、仕事中はメールチェックするしかありませんね。',
 '普通な人なら、料理を作ることはほとんどしかありません。何か特別な能力があるのかな？それとも仕事があるのかな？それ以外には何もできない気がするよ。',
 '普通の人は何か特別なスキルがなく、特殊な状況以外では何もできないという考えでは、少し狭いですね。しかり、基本的な日常生活のスキルしかありません。しかり、何かを始めるためのエネルギーと時間',
 'あんたは普通な人って想像してください。そんな人なら料理を作ることはほとんどありませんはずです。しかしこの一つだけはできるかもしれません。しかりあんたなら、お弁当を作るのは難しくないはずです。',
 'あんたは普通な人って想像してください。何をできるしかありませんな。']

In [12]:
# Responses might be better with grammar? More english junk, but the non-junk are shorter and more straight-forward
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using the かどうか grammar pattern"
user_input = "Ask me a quesiton in Japanese."
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "かどうか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [39]:
training_data = list()
training_size = 1000
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['あなたは今日学校に行きますか？',
 '日本の Seasons について、秋が好きかどうか教えてください。',
 'あなたは動物園に行きたいかどうか知りたいです。',
 '日本のどの都市に住んでいますか、東京かどうか？',
 'あなたは今日公園に行きますか行きますか？（あなたは今日公園に行きますか？行きますか？）',
 '日本のどの都市に住むかどうか聞かせてください。',
 '日本語でお聞きBecome a question using the かどうか grammar pattern. \n\nあなたは今日お絵かきをするかどうか知りたいです。',
 '日本語でお尋ねしますね。この質問には、「かどうか」を使用してみました。\n\n絵本の主人公は、新しい友達ができたかどうか知りたいですか？',
 'あなたは今日公園に行くかどうか知っていますか？',
 'あなたは今日学校に行きますか行かないか知りたいです。',
 'あなたは今日公園に行くかどうか教えてください。',
 'あなたはお気に入りのキャラクターがいるかどうか教えてください。',
 'あなたは動物園に行きたいかどうか知りたいです。',
 'あなたは今天気-goodkah（ですか？）',
 'あなたの favorite の本はなんかどうか教えてあげる？（あなたの好きな本は何か-knowing-？）',
 'あなたは今日公園で遊ぶかどうか教えてください。',
 'あなたは今日公園に行こうかといますか？',
 'あなたはりんごを食べますか、またはみかんを食べますか？',
 'あなたは猫が好きかどうか知りたいです。猫が好きですか？それとも好きじゃないですか？',
 'あなたはお絵かきが好きかどうか教えてください。',
 'あなたは今日学校にいくかどうか教えてください。',
 'あなたは今日公園に行きたいかどうか知っていますか？',
 'あなたは今日公園に行こうかと思いますか？',
 'あなたは今日公園に行きたいですか？行きたいかどうか教えてください。',
 'あなたは今日公園で遊ぶかどうか知っていますか？',
 'あなたは今日公園に行きますか行かないか知りたいです。',
 'あなたは今から本を読むかどうか教えてください。',
 'あなたは TODAYの朝起きてから学校に行く前に ドラゴンボールを見たかどうか 知ってい

In [41]:
import torch
import torch.nn as nn
from transformers.models.qwen2.modeling_qwen2 import Qwen2DecoderLayer
from transformers import Qwen2ForCausalLM, Qwen2Config
import transformers
from matplotlib import pyplot as plt
import gc

In [15]:
class IMDecoderLayer(nn.Module):
    mask = None
    vspace_to_emb = None
    emb_to_vspace = None

    def __init__(self, original_layer, emb_to_vspace, config):
        super().__init__()
        self.original_layer = original_layer

        if IMDecoderLayer.mask == None:
            IMDecoderLayer.mask = torch.zeros(config.vocab_size).to('cuda')

        if IMDecoderLayer.vspace_to_emb == None:
            IMDecoderLayer.vspace_to_emb =  nn.Linear(config.vocab_size, config.hidden_size).to('cuda')
        
        if IMDecoderLayer.emb_to_vspace == None:
            IMDecoderLayer.emb_to_vspace =  emb_to_vspace

    def forward(self, hidden_states, *args, **kwargs):
        hidden_states = self.original_layer(hidden_states, *args, **kwargs)
        hidden_states = hidden_states[0]

        # TODO: for inference, bypass this process on all tokens but the current one? Possible problem: maybe it will learn to rely on past tokens being allowed ones later in residual.
        # ^ maybe this process should only be done on last token if later note is a problem. If it isnt a problem, then it's preferable to do this b/c the layer will train faster.

        residual = hidden_states
        hidden_states = IMDecoderLayer.emb_to_vspace(residual)
        hidden_states *= IMDecoderLayer.mask
        hidden_states = IMDecoderLayer.vspace_to_emb(hidden_states)
        # hidden_states = (hidden_states + residual)/2 # NOTE: should also be normalization? -- avg: has to learn to recreate residual with affects added on -- add&norm: most things can be zero and only learns to affect some values
        hidden_states = hidden_states + residual # input layernorm means don't need to norm here? (and SHOULD norm after hidden state??)

        return (hidden_states,)

In [ ]:
MODEL_NAME = 'Qwen/Qwen2.5-0.5B-Instruct'

config = Qwen2Config.from_pretrained(MODEL_NAME)

model = Qwen2ForCausalLM.from_pretrained(MODEL_NAME).to('cuda')

# FREEZE existing model. Only the new layer in IMDecoderLayer will be trained
for param in model.parameters():
    param.requires_grad = False

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
optimizer = torch.optim.AdamW([p for p in IMDecoderLayer.vspace_to_emb.parameters() if p.requires_grad], lr=5e-5)
for i, _ in enumerate(model.model.layers):
    model.model.layers[i] = IMDecoderLayer(model.model.layers[i], model.lm_head, config)

print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x IMDecoderLayer(
        (original_layer): Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [42]:
for sample in training_data:
    try:
        tokens = tokenizer(sample, return_tensors='pt', padding=True)
        tokens = {k:v.to('cuda') for k,v in tokens.items()}
        outputs = model(**tokens, labels=tokens['input_ids'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    except torch.OutOfMemoryError:
            print(f'OOM on {i}')
            try: del tokens, outputs
            except NameError: pass
            gc.collect()



OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999
OOM on 999

In [50]:
tokens = tokenizer(['User: Write 3 example questions in Japanese.\nAssistant: '], return_tensors='pt', padding=True)
tokens = {k:v.to('cuda') for k,v in tokens.items()}

out = model.generate(
# out = model(
    **tokens,
    # labels=tokens['input_ids'],
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.001,
    do_sample=True,
    return_dict_in_generate=True,
    output_hidden_states=True
)
out_ids = out['sequences']
print(tokenizer.decode(out_ids[0]))

User: Write 3 example questions in Japanese.
Assistant: はい、教えて！ 1. 日本の名前を教えてください。 2. 今日食べることかどうか聞かに行きますか？ 3. 何時起きるか教えてください。 はい、ありがとう！<|endoftext|>
